In [1]:
import pandas as pd    

In [2]:
import networkx as nx

In [3]:
proscript = pd.read_json(path_or_buf="./proscript_v1a/dev.jsonl", lines=True)

## Dataset format

In [4]:
'''
list of dictionary: 
[
{topic: sentence,  
 text: list[script_list_A[sentence1, sentence2], script_list_B[sentence3, sentence4], ... ];
 }
]
'''

'\nlist of dictionary: \n[\n{topic: sentence,  \n text: list[script_list_A[sentence1, sentence2], script_list_B[sentence3, sentence4], ... ];\n }\n]\n'

## proscript

In [5]:
proscript.head(3)

,scenario,events,context,minutes,events_minutes,flatten_input_for_edge_prediction,flatten_input_for_script_generation,flatten_output_for_edge_prediction,flatten_output_for_script_generation,gold_edges_for_prediction
0,ride a train,"{'0': 'walk to train to ride', '1': 'ask to bu...",NONE,13.0,"{'0': 5.0, '1': 1.0, '2': 3.0, '3': 2.0, '4': ...",step0: walk to train to ride; step1: ask to bu...,You want to ride a train. How can you do this ...,step2 -> step4; step4 -> step1; step1 -> step3...,step0: decided to ride a train; step1: walk to...,"[0->6, 1->3, 2->4, 3->0, 4->1, 5->2]"
1,win the minor league baseball,"{'0': 'go to the locker room', '1': 'drive to ...",NONE,45.0,"{'0': 5.0, '1': 30.0, '2': 5.0, '3': 10.0, '4'...",step0: go to the locker room; step1: drive to ...,You want to win the minor league baseball. How...,step4 -> step1; step1 -> step2; step2 -> step0...,step0: decided to win the minor league basebal...,"[0->3, 1->2, 2->0, 3->6, 4->1, 5->4]"
2,catch a big marlin,"{'0': 'gather fishing tools', '1': 'drive to t...",NONE,120.0,"{'0': 15.0, '1': 45.0, '2': 5.0, '3': 5.0, '4'...",step0: gather fishing tools; step1: drive to t...,You want to catch a big marlin. How can you do...,step4 -> step3; step3 -> step0; step3 -> step6...,step0: decided to catch a big marlin; step1: l...,"[0->1, 1->5, 2->8, 3->0, 3->6, 4->3, 5->2, 6->..."


In [6]:
proscript

,scenario,events,context,minutes,events_minutes,flatten_input_for_edge_prediction,flatten_input_for_script_generation,flatten_output_for_edge_prediction,flatten_output_for_script_generation,gold_edges_for_prediction
0,ride a train,"{'0': 'walk to train to ride', '1': 'ask to bu...",NONE,13.0,"{'0': 5.0, '1': 1.0, '2': 3.0, '3': 2.0, '4': ...",step0: walk to train to ride; step1: ask to bu...,You want to ride a train. How can you do this ...,step2 -> step4; step4 -> step1; step1 -> step3...,step0: decided to ride a train; step1: walk to...,"[0->6, 1->3, 2->4, 3->0, 4->1, 5->2]"
1,win the minor league baseball,"{'0': 'go to the locker room', '1': 'drive to ...",NONE,45.0,"{'0': 5.0, '1': 30.0, '2': 5.0, '3': 10.0, '4'...",step0: go to the locker room; step1: drive to ...,You want to win the minor league baseball. How...,step4 -> step1; step1 -> step2; step2 -> step0...,step0: decided to win the minor league basebal...,"[0->3, 1->2, 2->0, 3->6, 4->1, 5->4]"
2,catch a big marlin,"{'0': 'gather fishing tools', '1': 'drive to t...",NONE,120.0,"{'0': 15.0, '1': 45.0, '2': 5.0, '3': 5.0, '4'...",step0: gather fishing tools; step1: drive to t...,You want to catch a big marlin. How can you do...,step4 -> step3; step3 -> step0; step3 -> step6...,step0: decided to catch a big marlin; step1: l...,"[0->1, 1->5, 2->8, 3->0, 3->6, 4->3, 5->2, 6->..."
3,make banana muffins,"{'0': 'preheat the oven', '1': 'mix the dry in...",NONE,60.0,"{'0': 0.5, '1': 5.0, '2': 5.0, '3': 10.0, '4':...",step0: preheat the oven; step1: mix the dry in...,You want to make banana muffins. How can you d...,step3 -> step2; step2 -> step1; step2 -> step5...,step0: decided to make banana muffins; step1: ...,"[0->6, 1->4, 2->1, 2->5, 3->2, 4->0, 5->4, 6->..."
4,eat some food,"{'0': 'Open the pizza packaging', '1': 'Drive ...",NONE,15.0,"{'0': 0.16666666666666602, '1': 4.0, '2': 4.0,...",step0: Open the pizza packaging; step1: Drive ...,You want to eat some food. How can you do this...,step2 -> step3; step3 -> step1; step1 -> step0...,step0: decided to eat some food; step1: Drive ...,"[0->4, 1->0, 2->3, 3->1, 4->6, 5->2]"
...,...,...,...,...,...,...,...,...,...,...
1080,stop and use the restroom before the show,"{'0': 'use the facilities', '1': 'walk into th...",walk in and leave coat at coat check,5.0,"{'0': 0.75, '1': 0.08333333333333301, '2': 0.0...",step0: use the facilities; step1: walk into th...,You want to stop and use the restroom before t...,step3 -> step1; step1 -> step2; step2 -> step0...,step0: walk in and leave coat at coat check; s...,"[0->4, 1->2, 2->0, 3->1, 4->6, 5->3]"
1081,take notes about the subject,"{'0': 'write with the pen on paper', '1': 'ope...",do research on the topic,30.0,"{'0': 5.0, '1': 0.066666666666666, '2': 0.05, ...",step0: write with the pen on paper; step1: ope...,You want to take notes about the subject. How ...,step2 -> step0; step1 -> step2; step1 -> step3...,step0: do research on the topic; step1: open t...,"[0->4, 1->2, 1->3, 2->0, 3->0, 4->6, 5->1]"
1082,walk to cashier with makeup selections.,"{'0': 'look through make up', '1': 'locate che...",shop for makeup,2.0,"{'0': 10.0, '1': 0.05, '2': 0.1666666666666660...",step0: look through make up; step1: locate che...,You want to walk to cashier with makeup select...,step0 -> step4; step4 -> step3; step3 -> step2...,step0: shop for makeup; step1: look through ma...,"[0->4, 1->6, 2->1, 3->2, 4->3, 5->0]"
1083,walk towards the car,{'0': 'wiggle doorknob to confirm door is lock...,lock the front door,1.0,"{'0': 0.08333333333333301, '1': 0.083333333333...",step0: wiggle doorknob to confirm door is lock...,You want to walk towards the car. How can you ...,step5 -> step2; step2 -> step0; step0 -> step4...,step0: lock the front door; step1: Remove keys...,"[0->4, 1->3, 2->0, 3->7, 4->1, 5->2, 6->5]"


In [7]:
flatten_for_generation = proscript.iloc[0,-2]

In [8]:
proscript.iloc[1,-2].split('; ')

['step0: decided to win the minor league baseball',
 'step1: get in the car',
 'step2: drive to the baseball game',
 'step3: park the car',
 'step4: go to the locker room',
 'step5: hype up teammates',
 'step6: win the minor league baseball',
 'step0 -> step1',
 'step1 -> step2',
 'step2 -> step3',
 'step3 -> step4',
 'step4 -> step5',
 'step5 -> step6']

In [9]:
proscript.iloc[2,-2]

'step0: decided to catch a big marlin; step1: look up local fishing spots; step2: choose a fishing spot; step3: gather fishing tools; step4: gather fishing supplies; step5: drive to the fishing spot; step6: set up at the fishing spot; step7: reel a big marlin in; step8: catch a big marlin; step0 -> step1; step1 -> step2; step2 -> step3; step2 -> step4; step3 -> step5; step4 -> step5; step5 -> step6; step6 -> step7; step7 -> step8'

In [10]:
step_list = proscript.iloc[2,-2].split('; ')

In [11]:
[each for each in step_list if not '->' in each]

['step0: decided to catch a big marlin',
 'step1: look up local fishing spots',
 'step2: choose a fishing spot',
 'step3: gather fishing tools',
 'step4: gather fishing supplies',
 'step5: drive to the fishing spot',
 'step6: set up at the fishing spot',
 'step7: reel a big marlin in',
 'step8: catch a big marlin']

In [12]:
[each.split(': ')[-1] for each in step_list if not '->' in each]

['decided to catch a big marlin',
 'look up local fishing spots',
 'choose a fishing spot',
 'gather fishing tools',
 'gather fishing supplies',
 'drive to the fishing spot',
 'set up at the fishing spot',
 'reel a big marlin in',
 'catch a big marlin']

In [13]:
def convert_int(one_tuple):
    return tuple(int(each) for each in one_tuple)

In [14]:
def find_all_paths(graph, start, path=[]):
    path = path + [start]
    paths = [path]
    if len(graph[start]) == 0:  # No neighbors
        # print(path)
        pass
    for node in graph[start]:
        newpaths = find_all_paths(graph, node, path)
        for newpath in newpaths:
            paths.append(newpath)
    return paths

def find_full_path(all_paths, end):
    return [each for each in all_paths if each[-1] == end]

In [15]:
def flatten_to_full_path(flatten_output_for_script_generation):
    step_list = flatten_output_for_script_generation.split('; ')
    step_ids = [each for each in step_list if '->' in each]
    step_content = [each.split(': ')[-1] for each in step_list if not '->' in each] # split with :, keep latter part

    return [step_content]

In [16]:
flatten_to_full_path(proscript.iloc[2,-2])

[['decided to catch a big marlin',
  'look up local fishing spots',
  'choose a fishing spot',
  'gather fishing tools',
  'gather fishing supplies',
  'drive to the fishing spot',
  'set up at the fishing spot',
  'reel a big marlin in',
  'catch a big marlin']]

In [17]:
flatten_to_full_path(proscript.iloc[0,-2])

[['decided to ride a train',
  'walk to ticket booth',
  'walk up to booth',
  'ask to buy ticket',
  'pay for ticket',
  'walk to train to ride',
  'ride a train']]

In [18]:
# # test purpose
# def find_full_paths_v(graph, start):
#     full = []
#     def find_all_paths(graph, start, path=[]):
#         path = path + [start]
#         paths = [path]
#         if len(graph[start]) == 0:  # No neighbors
#             # print(path)
#             full.append(path)
#         for node in graph[start]:
#             newpaths = find_all_paths(graph, node, path)
#             for newpath in newpaths:
#                 paths.append(newpath)
#         return paths
#     all_paths = find_all_paths(graph, start)
#     return full

In [19]:
all_proscript = [{'source': 'proscript', 'topic':proscript.iloc[i,0], 'text':flatten_to_full_path(proscript.iloc[i,-2])} for i in range(len(proscript))]

In [20]:
len(all_proscript)

1085

In [ ]:
# [len(each['text']) for each in all_proscript]

In [22]:
# all_proscript

## OMICS

In [23]:
import xml.etree.ElementTree as ET

In [24]:
import glob
import os
OMICS_files = glob.glob('./datas/OMICS/*.xml')

In [25]:
OMICS_files.sort()

In [26]:
OMICS_files

['./datas/OMICS/answer telephone.xml',
 './datas/OMICS/clean laundry.xml',
 './datas/OMICS/hear the door bell.xml',
 './datas/OMICS/iron clothes.xml',
 './datas/OMICS/make coffee.xml',
 './datas/OMICS/order fast food.xml',
 './datas/OMICS/restaurant.xml',
 './datas/OMICS/return food back.xml',
 './datas/OMICS/scrambled eggs.xml',
 './datas/OMICS/take bus.xml',
 './datas/OMICS/take shower.xml',
 './datas/OMICS/use microwave.xml',
 './datas/OMICS/use vending machine.xml']

In [27]:
def process_OMICS(one_file):
    # file name is event name, remove xml
    topic = os.path.basename(one_file)[:-4]
    tree = ET.parse(one_file)
    root = tree.getroot()    
    
    script_list = []
    # iterate the tree
    for child in root:
        # child:  Element 'script' --> {'id': '1'}
        sub_script_list = []
        for sub in child:
            # sub: Element 'item' --> {'slot': '1', 'text': 'find the phone .'}
            sub_script_list.append(sub.attrib['text'])
        script_list.append(sub_script_list)
        
    return {'source':'OMICS', 'topic': topic, 'text': script_list}

In [28]:
process_OMICS(OMICS_files[0])

{'source': 'OMICS',
 'topic': 'answer telephone',
 'text': [['find the phone .', 'pick up the phone .', 'say hello .'],
  ['locate phone .',
   'pick up receiver .',
   'place receiver to ear .',
   'say hello .'],
  ['get phone .',
   'pick up receiver .',
   'place receiver to ear/speaker .',
   'say hello .'],
  ['go to the phone .', 'pick it up .', 'say hello .'],
  ['hear the phone ring .', 'pick up the phone .', 'say hello .'],
  ['pick up the receiver .'],
  ['pick phone up .', 'push talk button .', 'answer phone .'],
  ['taking the phone in hand .', 'pressing answer button .'],
  ['move to phone .',
   'pick up receiver .',
   'say hello .',
   'ask who is calling .'],
  ['find the telephone .',
   'pick up the telephone receiver .',
   'speak a greeting into the receiver s microphone .'],
  ['find the phone .',
   'pick up the phone s receiver .',
   "speak a greeting such as 'hello' ."],
  ['walk to the phone .', 'pick up the receiver .', "say 'hello' ."],
  ['hearing the pho

In [29]:
all_OMICS = [process_OMICS(OMICS_files[i]) for i in range(len(OMICS_files))]

In [30]:
# all_OMICS

In [31]:
## pilot_esd

In [32]:
import glob
import os
pilot_esd_files = glob.glob('./datas/pilot_esd/*.xml')

In [33]:
pilot_esd_files

['./datas/pilot_esd/washing ones hair.pilot.xml',
 './datas/pilot_esd/going to a funeral.pilot.xml',
 './datas/pilot_esd/taking a bath.pilot.xml',
 './datas/pilot_esd/fyling in an airplane.pilot.xml',
 './datas/pilot_esd/baking a cake.pilot.xml',
 './datas/pilot_esd/borrowing a book from the library.pilot.xml',
 './datas/pilot_esd/renovating a room.pilot.xml',
 './datas/pilot_esd/going to the swimming pool.pilot.xml',
 './datas/pilot_esd/going to the dentist.pilot.xml',
 './datas/pilot_esd/riding on a bus.pilot.xml']

In [34]:
def process_pilot_esd(one_file):
    
    # file name is event name, remove .pilot.xml
    topic = os.path.basename(one_file)[:-10]
    tree = ET.parse(one_file)
    root = tree.getroot()    
    
    script_list = []
    # iterate the tree
    for child in root:
        # child:  Element 'scripts' --> {'id': '1', 'source': '1'}
        sub_script_list = []
        for sub in child:
            # sub: Element 'item' -->{'slot': '1', 'original': 'Go to the bath room'}
            sub_script_list.append(sub.attrib['original'])
        script_list.append(sub_script_list)

    return {'source':'pilot_esd', 'topic': topic, 'text': script_list}

In [35]:
len(process_pilot_esd(pilot_esd_files[0])['text'])

12

In [36]:
all_pilot_esd = [process_pilot_esd(pilot_esd_files[i]) for i in range(len(pilot_esd_files))]

In [37]:
# all_pilot_esd

In [38]:
## stories_xml

In [39]:
stories_xml_files = glob.glob('./datas/stories_xml/*.xml')

In [40]:
stories_xml_files

['./datas/stories_xml/clean_the_floor.xml',
 './datas/stories_xml/trash_an_object.xml',
 './datas/stories_xml/close_the_curtains.xml',
 './datas/stories_xml/pack_a_mailing_box.xml',
 './datas/stories_xml/greet_a_visitor.xml',
 './datas/stories_xml/move_furniture.xml',
 './datas/stories_xml/walk_the_dog.xml',
 './datas/stories_xml/get_food_from_refrigerator.xml',
 './datas/stories_xml/paint_a_wall.xml',
 './datas/stories_xml/turn_the_heat_off.xml',
 './datas/stories_xml/perform_research_on_specified_topic.xml',
 './datas/stories_xml/unclog_the_toilet.xml',
 './datas/stories_xml/get_mail.xml',
 './datas/stories_xml/feed_the_fish.xml',
 './datas/stories_xml/turn_music_system_on.xml',
 './datas/stories_xml/do_laundry.xml',
 './datas/stories_xml/draw_the_curtains.xml',
 './datas/stories_xml/find_the_time.xml',
 './datas/stories_xml/open_package.xml',
 './datas/stories_xml/watch_for_the_kids_to_come_home.xml',
 './datas/stories_xml/secure_all_exits.xml',
 './datas/stories_xml/find_an_object.

In [41]:
def process_stories_xml(one_file):
    # file name is event name, remove xml
    topic = os.path.basename(one_file)[:-4].replace('_',' ')
    tree = ET.parse(one_file)
    root = tree.getroot()    
    
    script_list = []
    # iterate the tree
    for child in root:
        # child:  Element 'script' --> {'id': '1'}
        sub_script_list = []
        for sub in child:
            # sub: Element 'item' --> {'slot': '1', 'text': 'find the phone .'}
            sub_script_list.append(sub.attrib['text'])
        script_list.append(sub_script_list)
        
    return {'source':'stories_xml', 'topic': topic, 'text': script_list}

In [42]:
process_stories_xml(stories_xml_files[0])

{'source': 'stories_xml',
 'topic': 'clean the floor',
 'text': [['find a bucket .',
   'fill the buket with water .',
   'find a mop .',
   'dunk the mop in the bucket .',
   'wipe the floor .'],
  ['get broom .',
   'sweep floor .',
   'put away broom .',
   'get mop .',
   'get water .',
   'mop floor .',
   'put away mop and water .'],
  ['make cleaning solvent using water and detergent .',
   'place solvent in bucket .'],
  ['get cleaning chemicals .',
   'get a bucket of water .',
   'get a mop .',
   'dip mop in water .',
   'spray chemicals on floor .',
   'move mop over the floor .'],
  ['take out the mop .',
   'take out the mop bucket .',
   'place water in the mop bucket .',
   'dip the mop in the bucket .',
   'run the mop over the floor .',
   'put mop away when the floor is clean .'],
  ['get mop .', 'wet mop .', 'mop the floor .', 'dry mop .'],
  ['pick up any large objects .',
   'sweep up any dust or small objects .',
   'wash the floor with a mop and water .',
   'le

In [43]:
all_stories = [process_stories_xml(stories_xml_files[i]) for i in range(len(stories_xml_files))]

In [44]:
# all_stories

In [45]:
## wiki how

In [46]:
import pandas as pd
wikihow1 = pd.read_json(path_or_buf="./SubeventWriter_wikihow/datasets/WikiHow_data_for_Seq2SeqLM/sentence_test_42.json", lines=True)

In [47]:
wikihow1

,id,title,original_title,subevents,source
0,52826,How to Make a Felt Heart Card?,How to Make a Felt Heart Card,"[Start by preparing the card., Measure the hea...",summarization
1,68152,How to Sell Jewelry at Flea Markets?,How to Sell Jewelry at Flea Markets,"[Explore flea markets in your area., Network w...",summarization
2,49263,How to Make Money Cooking in Runescape?,How to Make Money Cooking in Runescape,[Acknowledge that higher skill levels will ben...,summarization
3,975,How to Add Pictures to MySpace?,How to Add Pictures to MySpace,"[Log in to Myspace., Click on Photos below on ...",summarization
4,58220,How to Pay a California Highway Patrol Ticket ...,How to Pay a California Highway Patrol Ticket ...,"[Read the citation carefully., Schedule time.,...",summarization
...,...,...,...,...,...
4995,6443,How to Become Rare on Animal Jam?,How to Become Rare on Animal Jam2,"[Be around for Rare Monday items., Use rare it...",summarization
4996,6444,How to Become Rare on Animal Jam?,How to Become Rare on Animal Jam3,[Create a YouTube account if you're okay about...,summarization
4997,6445,How to Become Rare on Animal Jam?,How to Become Rare on Animal Jam4,"[Trade for what you want., Gather as many rare...",summarization
4998,6446,How to Become Rare on Animal Jam?,How to Become Rare on Animal Jam5,"[Enter giveaways., Go to famous jammers stream...",summarization


In [48]:
wikihow1.iloc[0,3]

['Start by preparing the card.',
 'Measure the hearts.',
 'Cut out the heart shapes.',
 'Assemble into a pile of hearts before transferring to the card.',
 'Transfer to the card and glue into place on the card.']

In [49]:
gb = wikihow1.groupby('title')  

In [50]:
list_of_group = [gb.get_group(x) for x in gb.groups]

In [51]:
list_of_group[7]

,id,title,original_title,subevents,source
2448,241,How to Accessorize Your Home With Family Photos?,How to Accessorize Your Home With Family Photos1,"[Group your photographs by color., Showcase a ...",summarization
2449,242,How to Accessorize Your Home With Family Photos?,How to Accessorize Your Home With Family Photos5,[Place a montage of photographs underneath a p...,summarization


In [52]:
list_of_group[7].iloc[0,-2]

['Group your photographs by color.',
 'Showcase a special event like a vacation.',
 'Choose photographs of a particular person or of a specific family group and create a special montage.']

In [53]:
list_of_group[7].iloc[1,-2]

['Place a montage of photographs underneath a piece of glass if you have a glass-topped table.',
 'Find a company that turns photographs into wallpaper.']

In [54]:
def process_wikihow(topic_df):
    topic = topic_df.iloc[0,1]
    script_list = [topic_df.iloc[i,-2] for i in range(len(topic_df))]
    return {'source':'wikihow', 'topic': topic, 'text': script_list}

In [55]:
process_wikihow(list_of_group[7])

{'source': 'wikihow',
 'topic': 'How to Accessorize Your Home With Family Photos?',
 'text': [['Group your photographs by color.',
   'Showcase a special event like a vacation.',
   'Choose photographs of a particular person or of a specific family group and create a special montage.'],
  ['Place a montage of photographs underneath a piece of glass if you have a glass-topped table.',
   'Find a company that turns photographs into wallpaper.']]}

In [56]:
wiki_how_all = [process_wikihow(list_of_group[i]) for i in range(len(list_of_group))]

In [57]:
# wiki_how_all

## all data

In [58]:
all_data = all_proscript + all_OMICS + all_pilot_esd + all_stories + wiki_how_all

In [59]:
len(all_data)

3783

In [60]:
import pickle

with open('Scripts_learning_dataset_proscript_use_flatten_generation.pickle', 'wb') as handle:
    pickle.dump(all_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Scripts_learning_dataset_proscript_use_flatten_generation.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(all_data == b)

True
